In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd



In [ ]:
# paramter set up 
s0=100
r=0.05
vol=0.10
T=2
noofsteps=100
noofpaths=10000

def monte_carlo(s0,r,T,vol,noofsteps,noofpaths):
    dt=T/noofsteps
    z=np.random.standard_normal((noofpaths,noofsteps))
    rets = (r - 0.5*vol**2)*dt + vol*z*np.sqrt(dt)
    cum_rets = np.cumsum(rets,axis=1)
    paths = np.exp(cum_rets)*s0
    paths = np.hstack([s0*np.ones((noofpaths,1)),paths])
    time = np.arange(0,noofsteps+1)*dt
    return paths,time
    
def American_option_pricer(s0,strike,vol,r,T,noofsteps,noofpaths,type):
    paths,time =monte_carlo(s0,r,T,vol,noofsteps,noofpaths)
    type = type.lower()
    dt= T/noofsteps
    excercise_matrix=np.zeros((noofpaths,noofsteps+1))
    cashflows = np.zeros((noofpaths,noofsteps+1))
    discount = np.exp(-r*dt)
    Excercise_payoff_immediate = np.maximum(paths-strike,0) if type=='call' else np.maximum(strike-paths,0)
    cashflows[:,-1] = Excercise_payoff_immediate[:,-1]
    excercise_matrix[:,-1] = Excercise_payoff_immediate[:,-1] > 0
    for t in range(noofsteps-1,0,-1):
                itm = Excercise_payoff_immediate[:,t]>0
                x=paths[itm,t]
                y= cashflows[itm,t+1]*discount
                if len(x)==0:
                    cashflows[:, t] = cashflows[:, t+1] * discount
                    continue
                coeffs = np.polyfit(x,y,2)
                continuation= np.polyval(coeffs,x)
                excercise = Excercise_payoff_immediate[itm,t] > continuation
                itm_indices = np.where(itm)[0]
                excercise_matrix[itm_indices[excercise], t] = 1
                cashflows[itm,t] = np.where(excercise,Excercise_payoff_immediate[itm,t],cashflows[itm,t+1]*discount)
                not_itm = ~itm
                cashflows[not_itm,t] = cashflows[not_itm,t]*discount
                price =  np.mean(cashflows[:,1]*discount)
    return price,excercise_matrix
    
paths,time=monte_carlo(s0,r,T,vol,noofsteps,noofpaths)
price,exercised=American_option_pricer(s0,100,vol,r,T,noofsteps,noofpaths,'put')


In [ ]:
# Importnant plots 

# plot_1 number of paths and simulation
N=50
idx = np.random.choice(paths.shape[0],N,replace=False)
plt.figure(figsize=(10,6))
plt.style.use('dark_background')
for i in idx:
    plt.plot(time,paths[i,:],color='cornflowerblue',alpha=0.5)
plt.axhline(100,0,color='orange')
plt.xlabel('Time',fontdict=dict(family='serif',weight='bold',color='yellow'))
plt.ylabel('Number of paths',fontdict=dict(family='serif',weight='bold',color='yellow'))
plt.title('Monte carlo simulated paths ',fontdict=dict(family='serif',weight='bold',color='yellow'))
plt.grid(color='Red',alpha=0.3)
plt.tight_layout()
plt.legend()
    
#plot 2 EXCERICSE BOUNDARY VISAULIZATION
# Ensure time is 1D with length paths.shape[1]


rows, cols = np.where(exercised == 1)  # indices of exercise
S_ex = paths[rows, cols]               # prices where exercise happened
t_ex = time[cols]                      # times of exercise

plt.figure(figsize=(9, 5))
plt.scatter(t_ex, S_ex, s=8, alpha=0.5, color='#FF5733', label='Exercise')
plt.title('American Option: Exercise Events Across Time',fontdict=dict(family='serif',weight='bold',color='yellow'))
plt.xlabel('Time',fontdict=dict(family='serif',weight='bold',color='yellow'))
plt.ylabel('Underlying Price at Exercise',fontdict=dict(family='serif',weight='bold',color='yellow'))
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()
